In [1]:
import numpy as np
import h5py
import pandas as pd
from itertools import islice
from collections import Counter
import re
from itertools import islice
from collections import Counter
from io import BytesIO

In [5]:
mouse = h5py.File('mouse_matrix_v11.h5', "r") # fot reading h5 dataframe
data=mouse.keys()
for key in mouse.keys():
    print(key)
for key in mouse['meta'].keys():
    print(key)
list_of_columns=mouse['meta']['genes'].keys()
print(list_of_columns)

data
meta
genes
info
samples
<KeysViewHDF5 ['chromosome', 'ensembl_gene_id', 'gene_biotype', 'gene_symbol', 'genes', 'start_position']>


In [6]:
with h5py.File('mouse_matrix_v11.h5', 'r') as f:
    for key in f["meta/genes"].keys():
        a = f["meta/genes/"][key]
        print(key)
        print(a[:])

chromosome
[b'missing' b'11' b'11' ... b'MT' b'MT' b'MT']
ensembl_gene_id
[b'missing' b'ENSMUSG00000007777' b'ENSMUSG00000043644' ...
 b'ENSMUSG00000065947' b'ENSMUSG00000064367' b'ENSMUSG00000064368']
gene_biotype
[b'missing' b'protein_coding' b'lncRNA' ... b'protein_coding'
 b'protein_coding' b'protein_coding']
gene_symbol
[b'0610007P14Rik' b'0610009B22Rik' b'0610009L18Rik' ... b'mt-Nd4l'
 b'mt-Nd5' b'mt-Nd6']
genes
[b'0610007P14Rik' b'0610009B22Rik' b'0610009L18Rik' ... b'mt-Nd4l'
 b'mt-Nd5' b'mt-Nd6']
start_position
[b'missing' b'51576213' b'120239504' ... b'9877' b'11742' b'13552']


Now every dataset from h5 file we transform into single table in pandas and then concatenate them into one table
import h5py
arch_file = "/home/clever/test_cache/counts/archs4/mouse_matrix_v11.h5"
with h5py.File(arch_file, 'r') as arch_7:
    for key in arch_7["meta/samples"].keys():
        a = arch_7["meta/samples/"][key]
        
        print(key)
        print(a[:])

In [18]:
with h5py.File('mouse_matrix_v11.h5', 'r') as f:
    data_set=f['meta']['samples']
    lister=['taxid_ch1','status','source_name_ch1','series_id','relation','readstotal','readsaligned','platform_id','organism_ch1', 'molecule_ch1','library_strategy','library_selection', 'library_source','instrument_model','extract_protocol_ch1', 'characteristics_ch1','data_processing','geo_accession','type','title']
    for i in lister:
        a=data_set[f'{i}']
        a=pd.DataFrame(a, columns=[f'{i}'])
        for col, dtype in a.dtypes.items():
            if dtype == np.object_:  # Only process byte object columns.
                a[col] = a[col].apply(lambda x: x.decode("utf-8"))
        table=pd.concat([table, a],sort=False, axis=1)

        
        

In [22]:
table

,taxid_ch1,status,source_name_ch1,series_id,relation,readstotal,readsaligned,platform_id,organism_ch1,molecule_ch1,...,library_strategy,library_selection,library_source,instrument_model,extract_protocol_ch1,characteristics_ch1,data_processing,geo_accession,type,title
0,b'10090',b'Public on Apr 07 2015',b'kidney cortex',b'GSE54390',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,39116064,32884497,GPL13112,b'Mus musculus',b'total RNA',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Thin outer slice of kidney cortex was immediat...,tissue: kidney cortex\tstrain: C57BL/6\tgender...,Reads were aligned to the genome using the Bur...,GSM1314260,SRA,TRACK 43-75 kidney 1
1,b'10090',b'Public on Jun 10 2016',b'corpus callosum',b'GSE75330',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,1136541,308433,GPL13112,b'Mus musculus',b'polyA RNA',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Selected CNS regions were dissected in 300µm t...,strain: PDGFRa-Cre\tage: p61\tSex: M\ttreatmen...,Read processing was performed as described (Is...,GSM2164318,SRA,C1-1772117-029-A02
2,b'10090',b'Public on Jun 10 2016',b'dorsal horn',b'GSE75330',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,514285,148668,GPL13112,b'Mus musculus',b'polyA RNA',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Selected CNS regions were dissected in 300µm t...,strain: C57BL6\tage: p24\tSex: ?\ttreatment: N...,Read processing was performed as described (Is...,GSM2161048,SRA,C1-1772072-242-F03
3,b'10090',b'Public on Jan 01 2016',b'Lung tissue',b'GSE64027',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,26804780,23355974,GPL13112,b'Mus musculus',b'total RNA',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Total RNA was extracted from pooled dissected ...,strain: A/J\tSex: female\ttissue: Lung\ttime p...,Illumina Casava1.8 software used for basecalli...,GSM1563082,SRA,20NNK_LPS_Sil_I3C9
4,b'10090',b'Public on Sep 10 2012',b'Cortex',b'GSE36025\tGSE49417\tGSE49847',b'Reanalyzed by: GSE80797\tSuperseded by: GSE9...,265978941,173619445,GPL13112,b'Mus musculus',b'polyA RNA',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,http://genome.ucsc.edu/cgi-bin/hgTrackUi?db=mm...,lab: CSHL-m\tlab description: Gingeras - Cold ...,http://genome.ucsc.edu/cgi-bin/hgTrackUi?db=mm...,GSM1000563,SRA,CSHL_RnaSeq_Cortex_adult-8wks (superseded by G...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501049,b'10090',b'Public on Sep 23 2021',b'transfected with shGAA lentivirus',b'GSE184555',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,13275033,12476567,GPL21626,b'Mus musculus',b'total RNA',...,RNA-Seq,cDNA,transcriptomic,NextSeq 550,RNA was extracted using a QIAGEN Rneasy miniki...,cell line: C2C12\tdifferentiation protocol: Di...,Raw sequence that passed CASAV 1.8P/F filter w...,GSM5592485,SRA,Fed_GAA 3
501050,b'10090',b'Public on Sep 23 2021',b'transfected with shscrambled',b'GSE184555',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,12754218,11967090,GPL21626,b'Mus musculus',b'total RNA',...,RNA-Seq,cDNA,transcriptomic,NextSeq 550,RNA was extracted using a QIAGEN Rneasy miniki...,cell line: C2C12\tdifferentiation protocol: Di...,Raw sequence that passed CASAV 1.8P/F filter w...,GSM5592486,SRA,Fed_SC 1
501051,b'10090',b'Public on Sep 23 2021',b'transfected with shscrambled',b'GSE184555',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,12243444,11535137,GPL21626,b'Mus musculus',b'total RNA',...,RNA-Seq,cDNA,transcriptomic,NextSeq 550,RNA was extracted using a QIAGEN Rneasy miniki...,cell line: C2C12\tdifferentiation protocol: Di...,Raw sequence that passed CASAV 1.8P/F filter w...,GSM5592487,SRA,Fed_SC 2
501052,b'10090',b'Public on Sep 23 2021',b'transfected with shscrambled',b'GSE184555',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,12163822,11405087,GPL21626,b'Mus musculus',b'total RNA',...,RNA-Seq,cDNA,transcriptomic,NextSeq 550,RNA was extracted using a QIAGEN Rneasy miniki...,cell line: C2C12\tdifferentiation protocol: Di..

In [21]:
table.to_csv('table.csv')

### Декодируем все строки из юникод в обычные (убираем b' ')

for col, dtype in table.dtypes.items():
    if dtype == np.object_:  # Only process byte object columns.
        table[col] = table[col].apply(lambda x: x.decode("utf-8"))
table.to_csv('table.csv')

In [23]:
print(table.head())

  taxid_ch1                    status     source_name_ch1  \
0  b'10090'  b'Public on Apr 07 2015'    b'kidney cortex'   
1  b'10090'  b'Public on Jun 10 2016'  b'corpus callosum'   
2  b'10090'  b'Public on Jun 10 2016'      b'dorsal horn'   
3  b'10090'  b'Public on Jan 01 2016'      b'Lung tissue'   
4  b'10090'  b'Public on Sep 10 2012'           b'Cortex'   

                         series_id  \
0                      b'GSE54390'   
1                      b'GSE75330'   
2                      b'GSE75330'   
3                      b'GSE64027'   
4  b'GSE36025\tGSE49417\tGSE49847'   

                                            relation  readstotal  \
0  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...    39116064   
1  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...     1136541   
2  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...      514285   
3  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...    26804780   
4  b'Reanalyzed by: GSE80797\tSuperseded by: GSE9...   265978941

In [24]:
print(geo_accession[28].decode("utf-8"))

GSM1849922


Let's delete rows with words - single cell in a column 'source_name_ch1'

In [ ]:
df = pd.read_csv('table.csv') # read table into pandas dataframe
discard=["single cell"]
del_single=df[~df.source_name_ch1.str.contains('|'.join(discard))] # ~ - все, кроме того что дальше - после нее
del_single.to_csv('del_single.csv')

/var/folders/xm/lz8thrrs1l5cks282q243ty40000gn/T/ipykernel_1162/442747323.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('table.csv') # read table into pandas dataframe


In [40]:
del_single

,Unnamed: 0,taxid_ch1,status,source_name_ch1,series_id,relation,readstotal,readsaligned,platform_id,organism_ch1,...,library_strategy.1,library_selection.1,library_source.1,instrument_model.1,extract_protocol_ch1.1,characteristics_ch1.1,data_processing.1,geo_accession.1,type.1,title.1
0,0,b'10090',b'Public on Apr 07 2015',b'kidney cortex',b'GSE54390',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,39116064,32884497,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Thin outer slice of kidney cortex was immediat...,tissue: kidney cortex\tstrain: C57BL/6\tgender...,Reads were aligned to the genome using the Bur...,GSM1314260,SRA,TRACK 43-75 kidney 1
1,1,b'10090',b'Public on Jun 10 2016',b'corpus callosum',b'GSE75330',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,1136541,308433,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Selected CNS regions were dissected in 300µm t...,strain: PDGFRa-Cre\tage: p61\tSex: M\ttreatmen...,Read processing was performed as described (Is...,GSM2164318,SRA,C1-1772117-029-A02
2,2,b'10090',b'Public on Jun 10 2016',b'dorsal horn',b'GSE75330',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,514285,148668,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Selected CNS regions were dissected in 300µm t...,strain: C57BL6\tage: p24\tSex: ?\ttreatment: N...,Read processing was performed as described (Is...,GSM2161048,SRA,C1-1772072-242-F03
3,3,b'10090',b'Public on Jan 01 2016',b'Lung tissue',b'GSE64027',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,26804780,23355974,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Total RNA was extracted from pooled dissected ...,strain: A/J\tSex: female\ttissue: Lung\ttime p...,Illumina Casava1.8 software used for basecalli...,GSM1563082,SRA,20NNK_LPS_Sil_I3C9
4,4,b'10090',b'Public on Sep 10 2012',b'Cortex',b'GSE36025\tGSE49417\tGSE49847',b'Reanalyzed by: GSE80797\tSuperseded by: GSE9...,265978941,173619445,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,http://genome.ucsc.edu/cgi-bin/hgTrackUi?db=mm...,lab: CSHL-m\tlab description: Gingeras - Cold ...,http://genome.ucsc.edu/cgi-bin/hgTrackUi?db=mm...,GSM1000563,SRA,CSHL_RnaSeq_Cortex_adult-8wks (superseded by G...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501049,501049,b'10090',b'Public on Sep 23 2021',b'transfected with shGAA lentivirus',b'GSE184555',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,13275033,12476567,GPL21626,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,NextSeq 550,RNA was extracted using a QIAGEN Rneasy miniki...,cell line: C2C12\tdifferentiation protocol: Di...,Raw sequence that passed CASAV 1.8P/F filter w...,GSM5592485,SRA,Fed_GAA 3
501050,501050,b'10090',b'Public on Sep 23 2021',b'transfected with shscrambled',b'GSE184555',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,12754218,11967090,GPL21626,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,NextSeq 550,RNA was extracted using a QIAGEN Rneasy miniki...,cell line: C2C12\tdifferentiation protocol: Di...,Raw sequence that passed CASAV 1.8P/F filter w...,GSM5592486,SRA,Fed_SC 1
501051,501051,b'10090',b'Public on Sep 23 2021',b'transfected with shscrambled',b'GSE184555',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,12243444,11535137,GPL21626,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,NextSeq 550,RNA was extracted using a QIAGEN Rneasy miniki...,cell line: C2C12\tdifferentiation protocol: Di...,Raw sequence that passed CASAV 1.8P/F filter w...,GSM5592487,SRA,Fed_SC 2
501052,501052,b'10090',b'Public on Sep 23 2021',b'transfected with shscrambled',b'GSE184555',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,12163822,11405087,GPL21626,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,NextSeq 550,RNA was extracted using a QIAGEN Rneasy miniki...,cell line: C2C12\tdifferentiation protocol: Di...,Raw sequence that passed CASAV 1.8P/F filter w...,GSM559248

In [27]:
print(len(df))
print(table.head())

501054
  taxid_ch1                    status     source_name_ch1  \
0  b'10090'  b'Public on Apr 07 2015'    b'kidney cortex'   
1  b'10090'  b'Public on Jun 10 2016'  b'corpus callosum'   
2  b'10090'  b'Public on Jun 10 2016'      b'dorsal horn'   
3  b'10090'  b'Public on Jan 01 2016'      b'Lung tissue'   
4  b'10090'  b'Public on Sep 10 2012'           b'Cortex'   

                         series_id  \
0                      b'GSE54390'   
1                      b'GSE75330'   
2                      b'GSE75330'   
3                      b'GSE64027'   
4  b'GSE36025\tGSE49417\tGSE49847'   

                                            relation  readstotal  \
0  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...    39116064   
1  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...     1136541   
2  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...      514285   
3  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...    26804780   
4  b'Reanalyzed by: GSE80797\tSuperseded by: GSE9...   26

Create a table with only immune cells

In [41]:
df=pd.read_csv('del_single.csv')
immune_cell= df[df['source_name_ch1'].str.contains('Innate lymphoid cell|CD8|CD4|macrophage|microglia|Th17|T cell|B cell', case=False)] # можно усложнить до регулярного выражения !

immune_cell.to_csv('immune_cell.csv')

/var/folders/xm/lz8thrrs1l5cks282q243ty40000gn/T/ipykernel_1162/1694858263.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('del_single.csv')


In [42]:
immune_cell

,Unnamed: 0.1,Unnamed: 0,taxid_ch1,status,source_name_ch1,series_id,relation,readstotal,readsaligned,platform_id,...,library_strategy.1,library_selection.1,library_source.1,instrument_model.1,extract_protocol_ch1.1,characteristics_ch1.1,data_processing.1,geo_accession.1,type.1,title.1
19,19,19,b'10090',b'Public on Jan 30 2015',b'Primary bone marrow-derived macrophages (BMDM)',b'GSE56122\tGSE56123',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,127650415,103088157,GPL13112,...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Total RNA was extracted from 5x106 cells using...,strain: BXH2/TyJ\tcell type: BMDM cells (7th d...,Reads were quality filtered according to the I...,GSM1356226,SRA,BXH2_LPS_UT
24,24,24,b'10090',b'Public on Mar 24 2015',b'Peritoneal exudate cells (adherent cells)',b'GSE55385',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,23911641,21984761,GPL13112,...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,TRIzol\tRNA libraries were prepared for sequen...,treatment: None\tstrain: C57Bl6cJ,rsem v1.2.1/bowtie v0.12.9 was used to align t...,GSM1335239,SRA,PEC macrophage -IL10 -LPS replicate 2
27,27,27,b'10090',b'Public on Mar 07 2016',b'E12_PS34CD45-',b'GSE54574',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,3258324,1377372,GPL13112,...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Cells were sorted by FACS. Single cells isolat...,strain: c57bl6\tage: embryonic day E12.5\tgeno...,Illumina Casava 1.8.2 software used for baseca...,GSM1890434,SRA,E12_PS34CD45-_34
28,28,28,b'10090',b'Public on Jul 15 2016',b'Pro B cells',b'GSE72018\tGSE72019',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,50911147,45076587,GPL13112,...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Female mouse C57BL/6 B cell populations were s...,strain: C57BL/6JNimr\ttissue: Bone marrow\tage...,fastq files were created with CASAVA 1.8.1\tSe...,GSM1849922,SRA,Bcell-pro-R5
38,39,39,b'10090',b'Public on Jun 20 2016',b'Follicular B cells',b'GSE73532\tGSE73534',b'SRA: https://www.ncbi.nlm.nih.gov/sra?term=S...,12791738,4090516,GPL13112,...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,For bone marrow subset purification we used B6...,strain: C57Bl/6\tage: 6-14 weeks\tgender: pool...,Raw data were aligned to the mouse reference g...,GSM1897413,SRA,FOB RNA-seq_r1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492669,500959,500959,b'10090',b'Public on Sep 10 2021',b'fibroblast cells',b'GSE183704',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,11131614,8544928,GPL19057,...,RNA-Seq,cDNA,transcriptomic,Illumina NextSeq 500,Qiazol (Qiagen) was added directly to PBS rins...,cell line: NIH3T3\tsirna: CFIm59 si2,Raw reads were preprocessed to remove any adap...,GSM5568100,SRA,CFIm59 si2 passage 8
492670,500960,500960,b'10090',b'Public on Sep 10 2021',b'fibroblast cells',b'GSE183704',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,11655915,8742565,GPL19057,...,RNA-Seq,cDNA,transcriptomic,Illumina NextSeq 500,Qiazol (Qiagen) was added directly to PBS rins...,cell line: NIH3T3\tsirna: CFIm68 si1,Raw reads were preprocessed to remove any adap...,GSM5568101,SRA,CFIm68 si1 passage 3
492671,500961,500961,b'10090',b'Public on Sep 10 2021',b'fibroblast cells',b'GSE183704',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,9687597,7220444,GPL19057,...,RNA-Seq,cDNA,transcriptomic,Illumina NextSeq 500,Qiazol (Qiagen) was added directly to PBS rins...,cell line: NIH3T3\tsirna: CFIm68 si1,Raw reads were preprocessed to remove any adap...,GSM5568102,SRA,CFIm68 si1 passage 4
492672,500962,500962,b'10090',b'Public on Sep 10 2021',b'fibroblast cells',b'GSE183704',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,11954279,8987427,GPL19057,...,RNA-Seq,cDNA,transcriptomic,Illumina NextSeq 500,Qiazol (Qiagen) was added directly to PBS rins...,cell line: NIH3T3\tsirna: CFIm68 si2,Raw reads were preprocessed to remove any adap...,GSM5568104,SRA,CFIm68 si2 passage 3


Create a table with uniqe values by cell type

In [43]:
df = pd.read_csv('del_single.csv')
cell_count=df.groupby('source_name_ch1')['platform_id'].value_counts() #count on two column basis
#cell_count=df['source_name_ch1'].value_counts() #create a table with count of uniq cell types
cell_count.to_csv('cell_count.csv') # put table into file .csv

/var/folders/xm/lz8thrrs1l5cks282q243ty40000gn/T/ipykernel_1162/1197267492.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('del_single.csv')


In [44]:
cell_count

source_name_ch1                                                         platform_id
b"2W-Salmonella-infected Peyer's patches, mesenteric lymph node cells"  GPL19057         6
b"3'-Seq in OPMD mouse model #1; age of 12 months"                      GPL13112         1
b"3'-Seq in OPMD mouse model #1; age of 20 weeks"                       GPL13112         1
b"3'-Seq in OPMD mouse model #2; age of 12 months"                      GPL13112         1
b"3'-Seq in OPMD mouse model #2; age of 20 weeks"                       GPL13112         1
                                                                                      ... 
b'young_naive_Mono_Saline'                                              GPL19057         5
b'zona incerta'                                                         GPL13112       187
b'zygote from Cxxc1fl/fl;Zp3-Cre female and wildtype male'              GPL21273         2
b'zygote from wildtype female and wildtype male'                        GPL21273         3
b'zygo

Create a table with uniqe values by platform id

In [45]:
df = pd.read_csv('del_single.csv')
platform_count=df.groupby('platform_id')['source_name_ch1'].value_counts() #count on two column basis
#cell_count=df['source_name_ch1'].value_counts() #create a table with count of uniq cell types
platform_count.to_csv('platform_count.csv') # put table into file .csv

/var/folders/xm/lz8thrrs1l5cks282q243ty40000gn/T/ipykernel_1162/137564493.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('del_single.csv')


In [46]:
platform_count

platform_id  source_name_ch1    
GPL13112     b'Brain'               7605
             b'hypothalamus'        3104
             b'Liver'               2751
             b'Embryo'              1846
             b'ventral midbrain'    1845
                                    ... 
GPL24247     b'wt-1'                   1
             b'wt-2'                   1
             b'wt-3'                   1
             b'wt-4'                   1
missing      b'missing'             2036
Name: source_name_ch1, Length: 38323, dtype: int64

Create a table with uniqe values by cell type - only immune cells

In [47]:
df = pd.read_csv('immune_cell.csv')
cell_count=df.groupby('source_name_ch1')['platform_id'].value_counts() #count on two column basis
#cell_count=df['source_name_ch1'].value_counts() #create a table with count of uniq cell types
cell_count.to_csv('cell_count2.csv') # put table into file .csv

In [48]:
cell_count

source_name_ch1                                                                          platform_id
b"Alveolar macrophage  from flu infected mouse's lung"                                   GPL18480        3
b"Bone marrow derived macrophage- stimulated 30'/DMSO treated"                           GPL13112       11
b"CD19+IgD-B220+IgM+ 3'Ek-/- immature B cells"                                           GPL13112        2
b"CD19+IgD-B220+IgM-CD25+ckit- 3'Ek-/- pre-B cells"                                      GPL13112        2
b"CD4+CD8+ double positive 3'Ek-/- T cells"                                              GPL13112        2
                                                                                                        ..
b'wild-type C57BL/6 mouse peritoneal macrophages'                                        GPL15103        6
b'wildtype ProB cells (B220+CD43+IgM-)'                                                  GPL13112        2
b'wt proB cells, Replicate1'               

Create a table with uniqe values by platform id - only immune cells

In [49]:
df = pd.read_csv('immune_cell.csv')
platform_count=df.groupby('platform_id')['source_name_ch1'].value_counts() #count on two column basis
#cell_count=df['source_name_ch1'].value_counts() #create a table with count of uniq cell types
platform_count.to_csv('platform_count2.csv') # put table into file .csv

In [50]:
platform_count

platform_id  source_name_ch1                                  
GPL13112     b'microglia'                                         1023
             b'bone marrow-derived macrophages'                    477
             b'Bone Marrow Derived Macrophages'                     90
             b'CD4+ T cells'                                        80
             b'Primary bone marrow-derived macrophages (BMDM)'      77
                                                                  ... 
GPL24247     b'infected macrophages'                                 1
             b'intestinal CD45+ cells'                               1
             b'peritoneal macrophage'                                1
             b'wt proB cells, Replicate1'                            1
             b'wt proB cells, Replicate2'                            1
Name: source_name_ch1, Length: 3141, dtype: int64

Let's change the last table. Remove very small values, type of cells into rows, platform id into columns


In [51]:
df= pd.read_csv('platform_count2.csv')
df.columns=['platform_id', 'source_name_ch1', 'counter'] # rename the columns heads
df=df[df.counter>10] # delete the values less than 10
table = pd.pivot_table(df, values='counter', index=['source_name_ch1'], columns=['platform_id'], aggfunc=np.sum) #create a table with platfom_id in columns
table.to_csv('platform_count3.csv') # put table into file .csv

In [52]:
table

platform_id,GPL13112,GPL15103,GPL17021,GPL18480,GPL19057,GPL20795,GPL21103,GPL21273,GPL21493,GPL21626,GPL23479,GPL24247
source_name_ch1,,,,,,,,,,,,
"b""Bone marrow derived macrophage- stimulated 30'/DMSO treated""",11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b'AV node macrophage',76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b'Act B cell',NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b'Activated CD4+ T cells',13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b'Activated T cells',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
b'thioglycolate-induced peritoneal macrophages',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,NaN,NaN,NaN
b'thioglycollate elicited macrophage cells',18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b'tumor infiltrating macrophage',NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN


In [2]:
df = pd.read_csv('del_single.csv')


/var/folders/xm/lz8thrrs1l5cks282q243ty40000gn/T/ipykernel_1184/787893585.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('del_single.csv')


Create a table with needing columns - макрофаги
air_quality = pd.concat([air_quality_pm25, air_quality_no2], axis=0) - сложение таблиц

In [54]:
wf = df[['geo_accession', 'platform_id', 'source_name_ch1']] #wf - working frame
macrophage= wf[wf['source_name_ch1'].str.contains('macrophage|macrophages', case=False)]
macrophage.insert(2, "Cell_type", 'macrophage')
conditions = [
    (macrophage['source_name_ch1'].str.contains('[B,m]one[-,\s]marrow', case=False, regex=True)),
    (macrophage['source_name_ch1'].str.contains('Peritoneal', case=False)),
    (macrophage['source_name_ch1'].str.contains('Alveolar|lung', case=False)),
    (macrophage['source_name_ch1'].str.contains('tumor|tumour', case=False)),
    (macrophage['source_name_ch1'].str.contains('Splenic', case=False)),
    (macrophage['source_name_ch1'].str.contains('hepatic', case=False)),
    (macrophage['source_name_ch1'].str.contains('sciatic nerve', case=False)),
    (macrophage['source_name_ch1'].str.contains('Phagocytic', case=False)),
    (macrophage['source_name_ch1'].str.contains('Peripheral', case=False)),
    (macrophage['source_name_ch1'].str.contains('muscle', case=False)),
    (macrophage['source_name_ch1'].str.contains('1 day after TBI', case=False)),
    (~macrophage['source_name_ch1'].str.contains('[B,m]one[-,\s]marrow|Peritoneal|Alveolar|tumor|tumour|Splenic|lung|hepatic|sciatic nerve|Phagocytic|Peripheral|muscle|1 day after TBI' , case=False))
]

values = ['Bone Marrow Derived macrophages', 'Peritoneal macrophage', 'Alveolar macrophage', 'tumor macrophage','Splenic macrophages','hepatic macrophages', 'sciatic nerve macrophages','Phagocytic macrophages', 'Peripheral macrophages','muscle macrophages','1 day after TBI macrophages', 'other macrophage']
macrophage.insert(3, "Classes", np.select(conditions, values))
macrophage.to_csv('macrophage.csv')


In [55]:
wf = df[['geo_accession', 'platform_id', 'source_name_ch1']] #wf - working frame
T_cells= wf[wf['source_name_ch1'].str.contains('/sT[-,\s)]cell[\s,s]|\sCD8[+,\s][\s]|[\s]CD4[+,\s][\s]', case=False)]
T_cells.insert(2, "Cell_type", 'T_cells')
conditions = [
    (T_cells['source_name_ch1'].str.contains('[\s]CD8[+,\s][\s]',case=False, regex=True)),
    (T_cells['source_name_ch1'].str.contains('[\s]CD4[+,\s][\s]',case=False, regex=True)),
    (T_cells['source_name_ch1'].str.contains('YFP+', case=False, regex=True)),
    (T_cells['source_name_ch1'].str.contains('V\xce\xb34', case=False)),
    (T_cells['source_name_ch1'].str.contains('Tumor-infiltrating', case=False)),
    (T_cells['source_name_ch1'].str.contains('Thymus|Thymic', case=False)),
    (T_cells['source_name_ch1'].str.contains('Splenic|spleen', case=False)),
    (~T_cells['source_name_ch1'].str.contains('[\s]CD8[+,\s]|[\s]CD4[+,\s][\s]|YFP+|V\xce\xb34|Tumor-infiltrating|Thymus|Thymic|Splenic|spleen', case=False, regex=True))
]
values = ['CD8+ T cells', 'CD4+ T cells', 'YFP+ T cells','V\xce\xb34 T cell', 'Tumor-infiltrating T cells', 'Thymus T cells', 'Splenic T cells', 'other T cells']
T_cells.insert(3, "Classes", np.select(conditions, values))
T_cells.to_csv('T_cells.csv')

In [56]:
wf = df[['geo_accession', 'platform_id', 'source_name_ch1']] #wf - working frame
B_cells= wf[wf['source_name_ch1'].str.contains('[\s]B[-, \s]cell[s,\s]', case=False, regex=True)]
B_cells.insert(2, "Cell_type", 'B_cells')
conditions = [
    (B_cells['source_name_ch1'].str.contains('splenic|from spleen|spleenic',case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('Germinal center',case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('Marginal zone' , case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('Follicular|folicular' , case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('B220+' , case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('Primary' , case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('NP+ GC', case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('immature', case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('CH12 ', case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('CD19+', case=False, regex=True)),
    (~B_cells['source_name_ch1'].str.contains('splenic|from spleen|spleenic|Germinal center|Marginal zone|Follicular|folicular|B220+|primary|NP+ GC|immature|CH12|CD19+ ' , case=False, regex=True))

]
values = ['splenic B cells', 'Germinal center B cells', 'Marginal zone B cells', 'Follicular B cells', 'B220+ B cells','Primary B cells','NP+ GC B cells','Immature B cells','CH12 line','CD19+ B cells', 'other B-cells']
B_cells.insert(3, "Classes", np.select(conditions, values))
B_cells.to_csv('B_cells.csv')

# Создадим большую таблицу со всеми типами и классами - great_table->great_table2

In [57]:
B_cells=pd.read_csv('B_cells.csv')
T_cells=pd.read_csv('T_cells.csv')
macrophage=pd.read_csv('macrophage.csv')
great_table = pd.concat([B_cells, T_cells, macrophage], axis=0)

great_table.to_csv('great_table.csv')

In [58]:
great_table

,Unnamed: 0,geo_accession,platform_id,Cell_type,Classes,source_name_ch1
0,28,GSM1849922,GPL13112,B_cells,other B-cells,b'Pro B cells'
1,38,GSM1897413,GPL13112,B_cells,Follicular B cells,b'Follicular B cells'
2,334,GSM1849954,GPL13112,B_cells,other B-cells,b'B1A B cells'
3,341,GSM1229014,GPL13112,B_cells,other B-cells,b'Dnmt-KO B-Cells'
4,570,GSM1234739,GPL13112,B_cells,Primary B cells,b'Primary B-cells'
...,...,...,...,...,...,...
6358,492513,GSM5550909,GPL19057,macrophage,other macrophage,b'Macrophages'
6359,492636,GSM5558220,GPL21626,macrophage,other macrophage,b'macrophages_WT'
6360,492637,GSM5558221,GPL21626,macrophage,other macrophage,b'macrophages_WT'
6361,492638,GSM5558224,GPL21626,macrophage,other macrophage,b'macrophages_Dnmt3a KO'


get indicis of lines with geo accession
indices[i]=indices[i-1] можно так 

In [9]:
great_table=pd.read_csv('great_table.csv')
df = pd.read_csv('table.csv')
res=df[df.geo_accession.isin(great_table.geo_accession)].reset_index(drop=False)
indices=res['index']
indices2=[]
for i in indices:
    f=i
    indices2.append(f)

/var/folders/xm/lz8thrrs1l5cks282q243ty40000gn/T/ipykernel_2926/3301459707.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('table.csv')


In [60]:
print(len(great_table))
print(len(indices))

9410
9410


In [91]:
df.head(20)

,Unnamed: 0,taxid_ch1,status,source_name_ch1,series_id,relation,readstotal,readsaligned,platform_id,organism_ch1,...,library_strategy.1,library_selection.1,library_source.1,instrument_model.1,extract_protocol_ch1.1,characteristics_ch1.1,data_processing.1,geo_accession.1,type.1,title.1
0,0,b'10090',b'Public on Apr 07 2015',b'kidney cortex',b'GSE54390',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,39116064,32884497,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Thin outer slice of kidney cortex was immediat...,tissue: kidney cortex\tstrain: C57BL/6\tgender...,Reads were aligned to the genome using the Bur...,GSM1314260,SRA,TRACK 43-75 kidney 1
1,1,b'10090',b'Public on Jun 10 2016',b'corpus callosum',b'GSE75330',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,1136541,308433,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Selected CNS regions were dissected in 300µm t...,strain: PDGFRa-Cre\tage: p61\tSex: M\ttreatmen...,Read processing was performed as described (Is...,GSM2164318,SRA,C1-1772117-029-A02
2,2,b'10090',b'Public on Jun 10 2016',b'dorsal horn',b'GSE75330',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,514285,148668,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Selected CNS regions were dissected in 300µm t...,strain: C57BL6\tage: p24\tSex: ?\ttreatment: N...,Read processing was performed as described (Is...,GSM2161048,SRA,C1-1772072-242-F03
3,3,b'10090',b'Public on Jan 01 2016',b'Lung tissue',b'GSE64027',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,26804780,23355974,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Total RNA was extracted from pooled dissected ...,strain: A/J\tSex: female\ttissue: Lung\ttime p...,Illumina Casava1.8 software used for basecalli...,GSM1563082,SRA,20NNK_LPS_Sil_I3C9
4,4,b'10090',b'Public on Sep 10 2012',b'Cortex',b'GSE36025\tGSE49417\tGSE49847',b'Reanalyzed by: GSE80797\tSuperseded by: GSE9...,265978941,173619445,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,http://genome.ucsc.edu/cgi-bin/hgTrackUi?db=mm...,lab: CSHL-m\tlab description: Gingeras - Cold ...,http://genome.ucsc.edu/cgi-bin/hgTrackUi?db=mm...,GSM1000563,SRA,CSHL_RnaSeq_Cortex_adult-8wks (superseded by G...
5,5,b'10090',b'Public on Oct 27 2016',b'E12.5 Single Cell',b'GSE87069',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,5607362,1929834,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Smart-seq2\tNextera XT DNA library preparation...,strain: C57BL/6 Heterozygous Lmx1a-EGFP\tdevel...,reads were aligned to the mm10 genome using de...,GSM2320580,SRA,F125_RNA-seq
6,6,b'10090',b'Public on Jun 10 2016',b'corpus callosum',b'GSE75330',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,1065010,473113,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Selected CNS regions were dissected in 300µm t...,strain: PDGFRa-Cre\tage: p61\tSex: M\ttreatmen...,Read processing was performed as described (Is...,GSM2164283,SRA,C1-1772117-028-A05
7,7,b'10090',b'Public on Feb 20 2015',b'sscortex',b'GSE60361',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,1211044,330539,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Mice were deeply anesthetized with ketamine/xy...,Sex: male\tage: P23,Read processing was performed as described (Is...,GSM1474588,SRA,SingleCellNo338
8,8,b'10090',b'Public on Oct 06 2016',b'ventral midbrain',b'GSE76381',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,663629,133904,GPL13112,b'Mus musculus',...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Human embryo ventral midbrain dissection was p...,tissue: ventral midbrain\tSex: pooled male and...,Read processing was performed as described (Is...,GSM2273393,SRA,1772075301_G03
9,9,b'10090',b'Public on Jun 10 2016',b'SN-VTA',b'GSE75330',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,456330,131980,GPL13112,b'Mus musculus',...,RNA-Seq,c

In [62]:
res

,index,Unnamed: 0,taxid_ch1,status,source_name_ch1,series_id,relation,readstotal,readsaligned,platform_id,...,library_strategy.1,library_selection.1,library_source.1,instrument_model.1,extract_protocol_ch1.1,characteristics_ch1.1,data_processing.1,geo_accession.1,type.1,title.1
0,19,19,b'10090',b'Public on Jan 30 2015',b'Primary bone marrow-derived macrophages (BMDM)',b'GSE56122\tGSE56123',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,127650415,103088157,GPL13112,...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Total RNA was extracted from 5x106 cells using...,strain: BXH2/TyJ\tcell type: BMDM cells (7th d...,Reads were quality filtered according to the I...,GSM1356226,SRA,BXH2_LPS_UT
1,28,28,b'10090',b'Public on Jul 15 2016',b'Pro B cells',b'GSE72018\tGSE72019',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,50911147,45076587,GPL13112,...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,Female mouse C57BL/6 B cell populations were s...,strain: C57BL/6JNimr\ttissue: Bone marrow\tage...,fastq files were created with CASAVA 1.8.1\tSe...,GSM1849922,SRA,Bcell-pro-R5
2,39,39,b'10090',b'Public on Jun 20 2016',b'Follicular B cells',b'GSE73532\tGSE73534',b'SRA: https://www.ncbi.nlm.nih.gov/sra?term=S...,12791738,4090516,GPL13112,...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,For bone marrow subset purification we used B6...,strain: C57Bl/6\tage: 6-14 weeks\tgender: pool...,Raw data were aligned to the mouse reference g...,GSM1897413,SRA,FOB RNA-seq_r1
3,65,65,b'10090',b'Public on Apr 07 2016',b'Mouse CD4+ cells 4h after activation',b'GSE52260',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,32010611,24778423,GPL13112,...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,RNA from human and mouse samples was extracted...,source: C57BL/6\tcell type: CD4+\tmouse biolog...,Illumina Casava1.8 software used for basecalli...,GSM1262088,SRA,C57BL/6 Th0 4h rep3
4,77,77,b'10090',b'Public on Jun 05 2013',b'bone marrow derived macrophages',b'GSE45914',b'SRA: https://www.ncbi.nlm.nih.gov/sra?term=S...,136512935,16000854,GPL13112,...,RNA-Seq,cDNA,transcriptomic,Illumina HiSeq 2000,ChIP-seq: Lysates were clarified from sonicate...,cell type: bone marrow derived macrophages (Re...,For ChIP-Seq and GRO-Seq samples (single end r...,GSM1119597,SRA,"RevErb WT BMDM, GRO-seq"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9405,500928,500928,b'10090',b'Public on Oct 07 2021',b'macrophages_Dnmt3a KO',b'GSE183480\tGSE184446',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,18404806,16283855,GPL21626,...,RNA-Seq,cDNA,transcriptomic,NextSeq 550,Total RNA from BMDMs were isolated using the R...,strain: C57BL/6\tgenotype/variation: Dnmt3a KO...,Raw sequencing fastq files were trimmed with T...,GSM5558224,SRA,RNA-seq Dnmt3a KO rep1 [D_KO_BMDM_9]
9406,500929,500929,b'10090',b'Public on Oct 07 2021',b'macrophages_Dnmt3a KO',b'GSE183480\tGSE184446',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,17712338,15570776,GPL21626,...,RNA-Seq,cDNA,transcriptomic,NextSeq 550,Total RNA from BMDMs were isolated using the R...,strain: C57BL/6\tgenotype/variation: Dnmt3a KO...,Raw sequencing fastq files were trimmed with T...,GSM5558226,SRA,RNA-seq Dnmt3a KO rep3 [D_KO_BMDM_11]
9407,500932,500932,b'10090',b'Public on Sep 10 2021',b'Lung-tumor infiltrating CD4+ T cells',b'GSE183544',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,25742942,23112530,GPL19057,...,RNA-Seq,cDNA,transcriptomic,Illumina NextSeq 500,Lung tissues from CD4/ALK4-KO or WT mice beari...,tissue: Lung tumor\tcell type: CD4+ T cells\tg...,Quality Control was performed at the fastq raw...,GSM5560174,SRA,Wild_Type_rep1
9408,500933,500933,b'10090',b'Public on Sep 10 2021',b'Lung-tumor infiltrating CD4+ T cells',b'GSE183544',b'BioSample: https://www.ncbi.nlm.nih.gov/bios...,29155148,26105830,GPL19057,...,RNA-Seq,cDNA,transcriptomic,Illumina NextSeq 500,Lung tissues from CD4/ALK4-KO or WT mice beari...,tissue: Lung tumor\tcell type: CD4+ T cells\tg...,Quality Co

In [61]:
indices

0           19
1           28
2           39
3           65
4           77
         ...  
9405    500928
9406    500929
9407    500932
9408    500933
9409    500934
Name: index, Length: 9410, dtype: int64

In [70]:
indices2[0:10]

[19, 28, 39, 65, 77, 107, 184, 241, 275, 280]

In [99]:
len(indices2)

9410

In [100]:
len(df)

501054

In [105]:
with h5py.File('mouse_matrix_v11.h5', 'r') as f:
    data_set=f['meta']['samples']['geo_accession']
    print(data_set[19])
    print(len(data_set))
    print(data_set.shape)

b'GSM1356226'
501054
(501054,)


In [7]:
with h5py.File('mouse_matrix_v11.h5', 'r') as f:
    data_set=f['data']['expression']
    print(data_set[19])
    print(len(data_set))
    print(len(data_set.shape))
    print(len(data_set[19]))
    print(data_set[:, 19])

[2 0 0 ... 0 1 2]
32544
2
501054
[  3228   1185    114 ...  22074 137790  55081]


create an expression table
'''
'''
with h5py.File('mouse_matrix_v11.h5', 'r') as f:
   data_set = f['data']['expression']
   data=data_set[::,indices2]
   expression = pd.DataFrame(data, columns=indices2)
   expression.to_csv('expression.csv')
   f.close()

In [11]:
with h5py.File('mouse_matrix_v11.h5', 'r') as f:
    data_set = f['data']['expression']
    data=data_set[::,indices2]
    expression = pd.DataFrame(data, columns=indices)
    expression.to_csv('expression3.csv')
    f.close()

In [12]:
print(len(expression))
print(expression.shape[1])

32544
9410


Заменим индексы колонок в таблице экспрессии на gsm
GSM1849922 - должно быть в позиции 28 - и это верно

In [13]:
wf=pd.read_csv('table.csv')
#df.drop(df.columns[[0]], axis = 1)
expression.columns = wf.loc[indices]['geo_accession']
print(expression.head())
expression.to_csv('expression4.csv')   

/var/folders/xm/lz8thrrs1l5cks282q243ty40000gn/T/ipykernel_2926/184194338.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  wf=pd.read_csv('table.csv')


geo_accession  GSM1356226  GSM1849922  GSM1897413  GSM1262088  GSM1119597  \
0                    3228        1274         114         810         440   
1                    1185         437          12          56         151   
2                     114          35          21          11         138   
3                    3548        3126          95         520         254   
4                    4075        1033          40         157         465   

geo_accession  GSM1533925  GSM1511235  GSM1808311  GSM1823626  GSM1713923  \
0                     117         500        1425         326        1121   
1                      78         136         757         237         200   
2                       4          46         101          15          44   
3                     112        1954        1063         558        1140   
4                     261          54        1834         948         441   

geo_accession  ...  GSM5550906  GSM5550908  GSM5550909  GSM5558220  \
0   

In [321]:
print(indices.head())
print(indices2[0:5])

0    19
1    28
2    39
3    65
4    77
Name: index, dtype: int64
[19, 28, 39, 65, 77]


In [322]:
df = pd.read_csv('table.csv')  
print(df.iloc[28])


Unnamed: 0                                                             28
taxid_ch1                                                        b'10090'
status                                           b'Public on Jul 15 2016'
source_name_ch1                                            b'Pro B cells'
series_id                                           b'GSE72018\tGSE72019'
relation                b'BioSample: https://www.ncbi.nlm.nih.gov/bios...
readstotal                                                       50911147
readsaligned                                                     45076587
platform_id                                                      GPL13112
organism_ch1                                              b'Mus musculus'
molecule_ch1                                                 b'total RNA'
library_strategy                                               b'RNA-Seq'
library_selection                                                 b'cDNA'
library_source                        

Теперь строки в таблице с экспрессией номер 4 нужно заменить названия строк на гены - indices 2

In [40]:
with h5py.File('mouse_matrix_v11.h5', 'r') as f:
    data_set = f['meta']['genes']['genes']
    data=data_set[::]
    names_lines = pd.DataFrame(data)
    ensembl_set=f['meta']['genes']['ensembl_gene_id']
    ensembl=ensembl_set[::]
    ensemble = pd.DataFrame(ensembl) 
    for col, dtype in names_lines.dtypes.items():
        if dtype == np.object_:  # Only process byte object columns.
            names_lines[col] = names_lines[col].apply(lambda x: x.decode("utf-8"))
    for col, dtype in ensemble.dtypes.items():
        if dtype == np.object_:  # Only process byte object columns.
            ensemble[col] = ensemble[col].apply(lambda x: x.decode("utf-8"))
    f.close()
print(names_lines.head())
print(ensemble.head())
print(len(ensemble))
print(len(names_lines))


               0
0  0610007P14Rik
1  0610009B22Rik
2  0610009L18Rik
3  0610009O20Rik
4  0610010F05Rik
                    0
0             missing
1  ENSMUSG00000007777
2  ENSMUSG00000043644
3             missing
4  ENSMUSG00000042208
32544
32544


In [41]:
expression=pd.read_csv('expression4.csv')
expression2 = pd.concat([ensemble, names_lines, expression], axis=1)
expression2.pop('Unnamed: 0')
expression2.to_csv('expression5.csv')   
print(expression2.head())


                    0              0  GSM1356226  GSM1849922  GSM1897413  \
0             missing  0610007P14Rik        3228        1274         114   
1  ENSMUSG00000007777  0610009B22Rik        1185         437          12   
2  ENSMUSG00000043644  0610009L18Rik         114          35          21   
3             missing  0610009O20Rik        3548        3126          95   
4  ENSMUSG00000042208  0610010F05Rik        4075        1033          40   

   GSM1262088  GSM1119597  GSM1533925  GSM1511235  GSM1808311  ...  \
0         810         440         117         500        1425  ...   
1          56         151          78         136         757  ...   
2          11         138           4          46         101  ...   
3         520         254         112        1954        1063  ...   
4         157         465         261          54        1834  ...   

   GSM5550906  GSM5550908  GSM5550909  GSM5558220  GSM5558221  GSM5558224  \
0        1629        1079         839        

In [42]:
f = pd.read_csv('expression5.csv')  
f.pop('Unnamed: 0')
f.rename(columns={'0': "ensemble", '0.1': "symbol"}, inplace=True)
f.to_csv('expression6.csv')   


In [43]:

print(f.head())

             ensemble         symbol  GSM1356226  GSM1849922  GSM1897413  \
0             missing  0610007P14Rik        3228        1274         114   
1  ENSMUSG00000007777  0610009B22Rik        1185         437          12   
2  ENSMUSG00000043644  0610009L18Rik         114          35          21   
3             missing  0610009O20Rik        3548        3126          95   
4  ENSMUSG00000042208  0610010F05Rik        4075        1033          40   

   GSM1262088  GSM1119597  GSM1533925  GSM1511235  GSM1808311  ...  \
0         810         440         117         500        1425  ...   
1          56         151          78         136         757  ...   
2          11         138           4          46         101  ...   
3         520         254         112        1954        1063  ...   
4         157         465         261          54        1834  ...   

   GSM5550906  GSM5550908  GSM5550909  GSM5558220  GSM5558221  GSM5558224  \
0        1629        1079         839        

 ### Создать таблицу - перечение с той что пришлют

In [3]:
dee2_exp=pd.read_csv('dee2_exp.csv')
print(dee2_exp.head())

   GSM4042993  GSM4042994  GSM4042995  GSM4042996  GSM4042997  GSM4042998  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   GSM4042999  GSM4043000  GSM4043001  GSM4043002  ...  GSM1234735  \
0           0           0           0           0  ...           0   
1           0           0           0           0  ...           1   
2           0           0           0           0  ...           0   
3           0           1           0           0  ...           0   
4           0           0           0           0  ...           0   

   GSM1234736  GSM1234737  GSM1234738  GSM1234739  GSM1234740  GSM1234741  \
0           0           0           0  

In [4]:
dee2_genes=pd.read_csv('dee2_genes.csv')
print(dee2_genes.head())
print(dee2_genes.shape[0])
print(dee2_exp.shape[1])

             ensemble         symbol
0  ENSMUSG00000102693  4933401J01Rik
1  ENSMUSG00000064842        Gm26206
2  ENSMUSG00000051951           Xkr4
3  ENSMUSG00000102851        Gm18956
4  ENSMUSG00000103377        Gm37180
52636
5558


In [5]:
dee2 = pd.concat([dee2_genes, dee2_exp], axis=1)
dee2.to_csv('dee.csv')   
print(dee2.head())


             ensemble         symbol  GSM4042993  GSM4042994  GSM4042995  \
0  ENSMUSG00000102693  4933401J01Rik           0           0           0   
1  ENSMUSG00000064842        Gm26206           0           0           0   
2  ENSMUSG00000051951           Xkr4           0           0           0   
3  ENSMUSG00000102851        Gm18956           0           0           0   
4  ENSMUSG00000103377        Gm37180           0           0           0   

   GSM4042996  GSM4042997  GSM4042998  GSM4042999  GSM4043000  ...  \
0           0           0           0           0           0  ...   
1           0           0           0           0           0  ...   
2           0           0           0           0           0  ...   
3           0           0           0           0           1  ...   
4           0           0           0           0           0  ...   

   GSM1234735  GSM1234736  GSM1234737  GSM1234738  GSM1234739  GSM1234740  \
0           0           0           0        

#### Проведем сортировку

In [3]:
exp = pd.read_csv('expression6.csv', encoding="utf8") # большая таблица
dee = pd.read_csv('dee.csv') #маленькая таблица
exp.pop('Unnamed: 0')
dee.pop('Unnamed: 0')
exp=exp.sort_values(by=['ensemble','symbol'])
dee=dee.sort_values(by=['ensemble','symbol'])
print(exp.shape[0])
print(dee.shape[0])

32544
52636


In [4]:
print(exp.shape[0])
print(exp.shape[1])
print(dee.shape[0])
print(dee.shape[1])

32544
9412
52636
5560


#### Из большой таблицы выделить ту часть, что соответствует маленькой

In [9]:
dee_cross=dee[dee['ensemble'].isin(exp['ensemble'])]
print(dee_cross.head())
print(dee_cross.shape[0])

                 ensemble symbol  GSM4042993  GSM4042994  GSM4042995  \
11893  ENSMUSG00000000001  Gnai3        3962        3807        2487   
8469   ENSMUSG00000000003   Pbsn           0           0           0   
46315  ENSMUSG00000000028  Cdc45        1254        1209        1144   
9726   ENSMUSG00000000037  Scml2           0           0           1   
39400  ENSMUSG00000000049   Apoh           0           0           3   

       GSM4042996  GSM4042997  GSM4042998  GSM4042999  GSM4043000  ...  \
11893        3122        3196        2420        5688        7300  ...   
8469            0           0           0           0           0  ...   
46315        1362        1310        1264        1109        1569  ...   
9726            1           0           0           2           0  ...   
39400           0           1           4           0           0  ...   

       GSM1234735  GSM1234736  GSM1234737  GSM1234738  GSM1234739  GSM1234740  \
11893        4077        4671        7293

In [10]:
exp_cross=exp[exp.ensemble.isin(dee_cross.ensemble)  ]
print(exp_cross.head())
print(exp_cross.shape[0])

                 ensemble symbol  GSM1356226  GSM1849922  GSM1897413  \
16843  ENSMUSG00000000001  Gnai3       17328        8597         507   
23599  ENSMUSG00000000003   Pbsn           0           0           0   
3402   ENSMUSG00000000028  Cdc45        1577        5479          27   
26959  ENSMUSG00000000037  Scml2          16          86           0   
1743   ENSMUSG00000000049   Apoh           1           0           0   

       GSM1262088  GSM1119597  GSM1533925  GSM1511235  GSM1808311  ...  \
16843        6175         536        1079        7421       10601  ...   
23599           0           0           0           0           0  ...   
3402           53         293          14         187         512  ...   
26959           7          50           0          10          32  ...   
1743            0           0           0           0           0  ...   

       GSM5550906  GSM5550908  GSM5550909  GSM5558220  GSM5558221  GSM5558224  \
16843       23609       13185       25213

1. Отсортировать положение столбцов
2. Перекрестить таблицы по списку столбцов

In [11]:
exp_cross = exp_cross[sorted(exp_cross.columns )]
dee_cross = dee_cross[sorted(dee_cross.columns )]
print(exp_cross.head())
print(dee_cross.head())

       GSM1026888  GSM1026889  GSM1026890  GSM1026891  GSM1026892  GSM1026893  \
16843        5567        5857        4467        5325        6059        5579   
23599           0           0           0           0           0           0   
3402           16          61          12          85         124         172   
26959           2           6           5           0           3          12   
1743            2           0           1           1           0           0   

       GSM1026894  GSM1026895  GSM1026896  GSM1026897  ...  GSM994541  \
16843        4753        5581        4557        3708  ...       9219   
23599           0           0           0           0  ...          0   
3402          117         188           7          29  ...       1913   
26959          10          16           1           3  ...         26   
1743            2           0           3           1  ...          0   

       GSM994542  GSM994543  GSM994544  GSM994545  GSM994546  GSM994547  \

In [12]:
col=dee_cross.columns
g=[]
for i in col:
    g.append(i)

In [13]:
exp_cross=exp_cross.loc[:, exp_cross.columns.intersection(g)]
col=exp_cross.columns
h=[]
for i in col:
    h.append(i)
dee_cross=dee_cross.loc[:, dee_cross.columns.intersection(h)]

In [14]:
print(exp_cross.head())
print(dee_cross.head())

       GSM1026888  GSM1026889  GSM1026890  GSM1026891  GSM1026892  GSM1026893  \
16843        5567        5857        4467        5325        6059        5579   
23599           0           0           0           0           0           0   
3402           16          61          12          85         124         172   
26959           2           6           5           0           3          12   
1743            2           0           1           1           0           0   

       GSM1026894  GSM1026895  GSM1026896  GSM1026897  ...  GSM951562  \
16843        4753        5581        4557        3708  ...       1090   
23599           0           0           0           0  ...          0   
3402          117         188           7          29  ...        123   
26959          10          16           1           3  ...          4   
1743            2           0           3           1  ...          0   

       GSM951563  GSM951564  GSM951565  GSM951566  GSM951567  GSM951568  \

In [15]:
exp_cross.to_csv('exp_cross.csv')
dee_cross.to_csv('dee_cross.csv')